In [ ]:
from pathlib import Path

from astropy.coordinates import EarthLocation
from astropy import units as u

from stellarphot import Camera, SourceListData
from stellarphot.photometry import multi_image_photometry
from stellarphot.gui_tools.photometry_widget_functions import PhotometrySettings

## You only need to change these settings if you switch camera/observatory/filter set up from your previous run

In [ ]:
##
## Setting defining the observatory and camera
##
feder_cg_16m = Camera(
    data_unit=u.adu,
    gain=1.5 * u.electron / u.adu,
    read_noise=10.0 * u.electron,
    dark_current=0.01 * u.electron / u.second,
    pixel_scale=0.563 * u.arcsec / u.pix,
    max_adu=50000 * u.adu,
)
feder_filters = {
    "up": "SU",
    "gp": "SG",
    "rp": "SR",
    "zp": "SZ",
    "ip": "SI",
    "V": "V",
    "B": "B",
}
feder_obs = EarthLocation(lat=46.86678, lon=-96.45328, height=311)

## Identify and Load the Previously Created Settings

Use the following cell to identify the previously created settings files.

In [ ]:
ps = PhotometrySettings()
ps.box

Once you have identified the three settings files above, execute the next cell to retrieve the information from the settings files.

In [ ]:
##
## Retrieve settings from the PhotometrySettings widget here
##

# The folder containing the images, a '.' means "the folder this notebook is in"
folder_with_images = ps.image_folder
image_directory = Path(folder_with_images)

# The file containing the photometry aperture information
source_file_name = ps.aperture_locations
sources = SourceListData.read(source_file_name)

# Retrieve the aperture and annulus settings
aperture_settings = ps.aperture_settings

# Retrieve the object name here
object_name = ps.object_name

## Adjust these Settings for this Data

In [ ]:
# The maximum allowed shift in an object position (in pixels) when comparing
# predicted positions to re-centroided positions
shift_tolerance = 6

# An estimate of the FWHM of the PSF in pixels.  Used for settings the size
# of the image cutouts used to estimate the FWHM of each photometry source.
fwhm_estimate = 5

# Set date to UTC date at the start of observations
date = ps.ifc.summary["date-obs"][0].split("T")[0]

# Enter the name you want to give the photometry data file (it should end in
# .ecsv).  This will be saved in the same directory as the images)
photometry_file = f"{object_name}-{date}.ecsv"

## Execute the Remaining Cells to Perform the Aperture Photometry and Save Results

In [ ]:
photometry_table = multi_image_photometry(
    image_directory,
    object_name,
    sources,
    feder_cg_16m,
    feder_obs,
    aperture_settings,
    shift_tolerance,
    fwhm_estimate,
    include_dig_noise=True,
    reject_too_close=True,
    reject_background_outliers=True,
    passband_map=feder_filters,
    fwhm_by_fit=False,
)

In [ ]:
photometry_table.write(photometry_file)

In [ ]:
photometry_table